In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np

import warnings

# Ignore FutureWarning and UserWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# We used a short compression function that improves the ram usage by downcasting datatypes that are too high (so no loss in information) and stored 
# it afterwards as "ram_compressed_dataset.pkl":

#train['ID_engine'] = train['ID_engine'].astype('category')
#train['cyl'] = train['cyl'].astype('uint8')
#train['time'] = train['time'].astype('uint16')
#train['load'] = train['load'].astype('float32')
#train['ign_time'] = train['ign_time'].astype('float32')
#train['knock_control'] = train['knock_control'].astype('float32')
#train['knock_signal'] = train['knock_signal'].astype('float32')
#train['noise_signal'] = train['noise_signal'].astype('float32')
#train['temp_exh'] = train['temp_exh'].astype('float32')
#train['case'] = train["case"].replace({"A": 1, "B": 2, "0": 0}).astype('float16')
#train.to_pickle('ram_compressed_dataset.pkl')

train = pd.read_pickle('/input/lec-pickled/ram_compressed_dataset.pkl') 
train

# Train Test Split

In [ ]:
import random
random.seed(41)

# A train/validation split of 90/10 but by Engine to not leak Engine level information

val_stations = random.sample(list(train["ID_engine"].unique()), int(len(train["ID_engine"].unique()) * 0.1))
val = train[train["ID_engine"].isin(val_stations)]
val = val.dropna(axis='rows')
val

train_stations = [obj for obj in list(train["ID_engine"].unique()) if obj not in val_stations]
train = train[train["ID_engine"].isin(train_stations)]
train = train.dropna(axis='rows')
train

# Feature Engeneering

In [ ]:
# Slight Downsampling of the majority class (class 0)
# We ditch some of the eninges that contain only cylinder with class 0  

grouped = train.groupby(["ID_engine"], observed=True).max()
stations_without_case = grouped[grouped["case"] == 0].index
stations_with_case = grouped[grouped["case"] != 0].index

stations_without_case_to_keep = np.random.choice(list(stations_without_case), size=25, replace=False) # keep only 25 stations where all cylinder are always class 0
stations_to_keep = list(stations_with_case) + list(stations_without_case_to_keep)
print(stations_to_keep)
len(stations_to_keep)

train = train[train["ID_engine"].isin(stations_to_keep)]

del grouped, stations_without_case, stations_with_case, stations_to_keep # del object to safe up some RAM
train

In [ ]:
# We set these columns as target columns for our feature engineering 
target_columns = ["knock_control", "knock_signal", "noise_signal", "temp_exh", "ign_time"]

In [ ]:
# Compute the difference between my cylinder and the mean cylinder (average of the respective engine) 

def add_difference_to_mean_cylinder(data, time_station_mean_target_columns):
    station_mean = data.groupby(['ID_engine', 'time'], observed=True)[time_station_mean_target_columns].mean().reset_index()
    data = data.merge(station_mean, on=['ID_engine', 'time'], suffixes=('', '_mean'))

    mean_columns = [f'{column}_mean' for column in time_station_mean_target_columns]
    diff_to_mean_columns = [f'{column}_diff_to_mean' for column in time_station_mean_target_columns]

    for diff_mean, target, mean_c in zip(diff_to_mean_columns, time_station_mean_target_columns, mean_columns):
        data[diff_mean] = data[target] - data[mean_c]

    data.drop(columns=mean_columns, inplace=True) # Drop mean column, we only want difference to mean columns
    return data, mean_columns, diff_to_mean_columns
        
time_station_mean_target_columns = target_columns 
train, mean_columns, diff_to_mean_columns =  add_difference_to_mean_cylinder(train, time_station_mean_target_columns)
train # train now contains the difference to the engine mean feature

In [ ]:
# Add a rolling mean over the target features and the difference to engine mean feature

def add_windows(data, columns_for_rolling_mean):
    window_sizes = [30]

    # Compute the rolling mean and add as new columns to the DataFrame
    for window_size in window_sizes:
        for column in columns_for_rolling_mean:
            data[f'{column}_rolling_mean_{window_size}'] = data[column].rolling(window=window_size, min_periods=1).mean()
            #data[f'{column}_rolling_std_{window_size}'] = data[column].rolling(window=window_size, min_periods=1).std()
            
    return data

columns_for_rolling_mean = target_columns + diff_to_mean_columns
train = add_windows(train, columns_for_rolling_mean)
train

In [ ]:
print(len(list(train.columns)))
train.columns

# Training

In [ ]:
non_included_features = ["case", "ID_engine", "cyl", ] # Features we want to exclude during training

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import xgboost as xgb
import time
from sklearn.metrics import matthews_corrcoef, hamming_loss
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

model = xgb.XGBClassifier(n_estimators=100, n_jobs=-1)

model.fit(train.drop(non_included_features, axis=1), train["case"])

del train

In [ ]:
import joblib
model_filename = "xgboost_model.joblib"
joblib.dump(model, model_filename) # save the model

In [ ]:
val, _, _ = add_difference_to_mean_cylinder(val, time_station_mean_target_columns)
val = add_windows(val, columns_for_rolling_mean)

val

In [ ]:
val["predictions"] = model.predict(val.drop(non_included_features, axis=1))

In [ ]:
# Function to calculate the longest sequence of consecutive appearances of class 1 or 2

def longest_consecutive(sequence, value):
    max_count = 0
    current_count = 0

    for num in sequence:
        if num == value:
            current_count += 1
            max_count = max(max_count, current_count)
        else:
            current_count = 0

    return max_count

# Group by "ID_engine" and "cyl" and calculate longest consecutive sequence of 1 or 2 in "predictions"
#val.groupby(["ID_engine", "cyl"], observed=True)["predictions"].apply(lambda x: longest_consecutive(x, 1))
#val.groupby(["ID_engine", "cyl"], observed=True)["predictions"].apply(lambda x: longest_consecutive(x, 2))

In [ ]:
# Post processing: setting predictions on cylinder level where class 1/2 appears less than 5 times to class 0 (reduce False Positives for Event Classification) 

def custom_function(group):
    if (group['predictions'] == 1).sum() < 5:
        group.loc[group['predictions'] == 1, 'predictions'] = 0
    if (group['predictions'] == 2).sum() < 5:
        group.loc[group['predictions'] == 2, 'predictions'] = 0
        
    return group

val_grouped = val.groupby(["ID_engine", "cyl"], observed=True).apply(custom_function)

val_grouped.reset_index(drop=True, inplace=True)
val = val_grouped
val

In [ ]:
# Scores and visualisation of the validation set predictions

def calculate_and_print_scores(val):
    stats_df = val.groupby(["ID_engine", "cyl"], observed=True).apply(lambda group: pd.Series({
        "case_0": (group["case"] == 0).sum(),
        "case_1": (group["case"] == 1).sum(),
        "case_2": (group["case"] == 2).sum(),
        "pred_0": (group["predictions"] == 0).sum(),
        "pred_1": (group["predictions"] == 1).sum(),
        "pred_2": (group["predictions"] == 2).sum(),
        "con_1": longest_consecutive(group["predictions"], 1),
        "con_2": longest_consecutive(group["predictions"], 2)
    })).reset_index()

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
        print(stats_df.groupby(["ID_engine", "cyl"], observed=True).max())
    
    
    val_by_cyl = val.groupby(["ID_engine", "cyl"], observed=True).max()
    binary_mcc = matthews_corrcoef(val_by_cyl["predictions"] != 0, val_by_cyl["case"] != 0) 

    # Calculate the multiclass Matthews correlation coefficient (MCC)
    multiclass_mcc = matthews_corrcoef(val_by_cyl["predictions"], val_by_cyl["case"]) 

    # Calculate the normalized Hamming distance
    hamming_distance = hamming_loss(val["predictions"], val['case']) 

    # Weighted average calculation
    overall_score = (0.6 * binary_mcc) + (0.2 * multiclass_mcc) + (0.2 *  (1 - hamming_distance))

    # Print the evaluation metrics
    print("Binary MCC:", binary_mcc)
    print("Multiclass MCC:", multiclass_mcc)
    print("Normalized Hamming Distance:", hamming_distance)
    print("Overall Score:", overall_score)
    
    cm = confusion_matrix(val_by_cyl['case'], val_by_cyl["predictions"])
    confusion_df = pd.DataFrame(cm, index=['True 0', 'True 1', 'True 2'], columns=['Pred 0', 'Pred 1', 'Pred 2'])

    # Here, we display the confusion matrix on Event Classification Level
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_df, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Class")
    plt.ylabel("True Class")
    plt.title("Confusion Matrix")
    plt.show()
    
    cm = confusion_matrix(val['case'], val["predictions"])
    confusion_df = pd.DataFrame(cm, index=['True 0', 'True 1', 'True 2'], columns=['Pred 0', 'Pred 1', 'Pred 2'])

    # Here, we display the confusion matrix on per timestep prediction level
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_df, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Class")
    plt.ylabel("True Class")
    plt.title("Confusion Matrix")
    plt.show()
    
    return binary_mcc, multiclass_mcc, hamming_distance, overall_score

calculate_and_print_scores(val)